# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
# The result for my linkedin page is empty, server side rendering ? javascirpt ? security ? No links?
# Potential solution is being talked in the previous lecture
# yifan = Website("https://www.linkedin.com/in/yifan-wei-a1576882")
# yifan.links

ed = Website("https://edwarddonner.com")
print(ed.get_contents())
print(ed.links)

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of pr

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/meta-llama/Llama-3.3-70B-Instruct',
 '/Datou1111/shou_xin',
 '/tencent/HunyuanVideo',
 '/black-forest-labs/FLUX.1-dev',
 '/CohereForAI/c4ai-command-r7b-12-2024',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/multimodalart/flux-style-shaping',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/ginipick/FLUXllama',
 '/spaces',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/CohereForAI/Global-MMLU',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets/aiqtech/kolaw',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/d

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'join page', 'url': 'https://huggingface.co/join'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
meta-llama/Llama-3.3-70B-Instruct
Updated
5 days ago
•
147k
•
1.03k
Datou1111/shou_xin
Updated
7 days ago
•
15.3k
•
411
tencent/HunyuanVideo
Updated
8 days ago
•
4.39k
•
1.04k
black-forest-labs/FLUX.1-dev
Upda

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nmeta-llama/Llama-3.3-70B-Instruct\nUpdated\n5 days ago\n•\n147k\n•\n1.03k\nDatou1111/shou_xin\nUpdated\n7 days ago\n•\n15.3k\n•\n411\ntencent/HunyuanVideo\nUpdated\n8 days ago\n•\n4.39k\n•\n1.04k\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16\n•\n1.36M\n•\n7.28k\nCohereForAI/c4ai-command-r7b-12-2024\nUpdated\n1 day ago\n•\n1.2k\n•\n185\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.35k\n🏢\nTRELLIS\nScalable and Versatile 3D Gene

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'company page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog', 'url': 'https://huggingface.com/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# 🤗 Hugging Face: The AI Community Building the Future! 🤖

---

**Welcome to Hugging Face**, where you can have a machine learning experience that’s as warm and snuggly as a cozy blanket… but with a whole lot more code! 

### Who Are We?
We are a community of brilliant minds and curious data enthusiasts determined to take machine learning from "meh" to "WOW!" Think of us as the social club for AI nerds, where everyone leaves smarter than when they came (and some also leave with questionable puns).

### What Do We Offer?
- **Models, Models, Models!**  
  Browse through over **400k+ models**! Whether you want to fine-tune your own or borrow the brilliant minds of others, we've got the options! From the latest and greatest like _Llama-3.3-70B-Instruct_ to the under-appreciated gems (hey, every model deserves a little love).

- **Datasets Galore!**  
  Love data? We've got **100k+ datasets**! From images of cats in spacesuits to audio of squirrels trying to read Shakespeare, we have it all! Just remember, if it squeaks or chirps, you might want to check it twice before using it for your model. 

- **Spaces for Creativity**  
  Our **Spaces** allow you to run applications that are as wild and creative as your imagination. Create optical illusions or even try-on virtual clothes! Who knew AI could help with your wardrobe choices? 

---

### Join Us!
At **Hugging Face**, we believe in collaboration and innovation. Our culture can best be described as:
- **As Fun as a Data Party**: We work hard but know how to giggle over some neural network mishaps.
- **Open as an Open Source Project**: If you bring the code, we’ll bring the snacks (virtual or otherwise).
- **Supportive Like a Great Friend**: Whether you’re a seasoned ML pro or just curious about AI, we all lift each other up. Think of us as a friend who always carves out time to help you debug your life.

---

### Who Loves Us?
More than **50,000 organizations** love using Hugging Face, including giants like Google, Microsoft, and even Grammarly (because who doesn’t need an AI wingman?). They trust us to keep their AI dreams alive. They might not hug back, but they sure appreciate a good model!

---

### Careers at Hugging Face:
Are you looking for a career that combines your love for AI with a community that celebrates your inner geek? **Join us!**
- **Job Perks**: Flexible hours, remote work, and an office pet (currently unsure if it's a cat, dog, or sentient code).
- **Opportunity for Growth**: You’ll learn faster than a toddler on a sugar rush!
- **Be Part of Something Bigger**: Help us build out the next big thing in AI… or at least make those awkward Zoom meetings a little less awkward!

---

So, whether you're an **investor** looking for the next big opportunity, a **customer** seeking cutting-edge AI solutions, or a **recruit** ready to join the most fun-loving AI gang around, come discover the joy of Hugging Face!

### Connect with Us
- 🌐 Website: [huggingface.co](https://huggingface.co)
- 🐦 Twitter: [@HuggingFace](https://twitter.com/huggingface)
- 👾 Discord: Join our friendly AI playground!

---

🤗 **Hugging Face** - Where AI Meets Cuddly Innovation!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/brand'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'jobs page', 'url': 'https://huggingface.co/join'}]}



# Welcome to Hugging Face!
### - The AI Community Building the Future (and Maybe Your Next AI Mood Ring)

---

## Who Are We?
At Hugging Face, we’re not just a bunch of data nerds holed up in a room with some magical algorithms (well, okay, maybe a little). We’re a vibrant **community** of AI enthusiasts dedicated to building the future, one dataset at a time while trying to figure out how to program humility into machines!

### What Do We Do?
- **Models**: Whether it's Llama-3.3-70B-Instruct (yep, that’s a mouthful) or a plethora of innovative applications, we’re making machine learning as accessible as trying to explain how you lost three hours to cat videos on the internet.
  
- **Datasets**: With over **100k datasets**, we’ve got more info than a squirrel at a nut factory. Perfect for all your data-hoarding needs!

- **Spaces**: Why do all the fun stuff on your own? Use our **Spaces** to show off your 3D generation skills, or engage in that optical illusion project you’ve got bubbling in the back of your mind. 

---

## Who's Using Us?
Join **50,000+ organizations** who have already discovered that building AI doesn't have to be an existential crisis. Major players like **Google**, **Microsoft**, and even **Grammarly** are part of our family. Yes, English is hard; that’s why they need us!

---

## Join the Culture!
At Hugging Face, we pride ourselves on a **collaborative culture** that embraces laughter, creativity, and the occasional debate over whether a hot dog is a sandwich. 

### Career Opportunities
Looking for a career where you can hug trees and code? We have positions ranging from **ML engineers** to **data wranglers**. We don't require you to know everything—just a passion for AI and a confidently apologetic expression when you break something! 

**Perks include:**
- A supportive environment that feels like family… minus the awkward Thanksgiving dinner conversations.
- Work-from-happy-places policies (aka our flexible work arrangements).

---

## How to Get Involved
- Sign up! Create and share your work on a platform with **400k+ models** (and counting)! 
- Check out our *Enterprise Solutions* that are so advanced even your grandma might be interested (just kidding, but you get the point).

---

### Ready to Create?
Come on over, and let’s make some magic! Whether you're an AI wizard or just looking for a way to impress your relatives at the next family gathering, Hugging Face is here to turn your dreams into datasets—one hug at a time!

**So… What are you waiting for?** 
*Sign up today!*  


---



In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("NVIDIA", "https://www.nvidia.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.nvidia.com/en-us/about-nvidia/'}, {'type': 'careers page', 'url': 'https://www.nvidia.com/en-us/about-nvidia/careers/'}, {'type': 'investor relations', 'url': 'https://investor.nvidia.com/home/default.aspx'}, {'type': 'company history', 'url': 'https://images.nvidia.com/pdf/NVIDIA-Story.pdf'}, {'type': 'executive insights', 'url': 'https://www.nvidia.com/en-us/executive-insights/'}]}


ChunkedEncodingError: ('Connection broken: IncompleteRead(170 bytes read, 18664049 more expected)', IncompleteRead(170 bytes read, 18664049 more expected))

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>